In [ ]:
# always check your python version
!python --version

In [ ]:
# AWS SDK for Python, -q flag is for quiet
%pip install -q boto3

In [ ]:
# always check your boto3 version
%pip show boto3

In [ ]:
!sudo apt-get install sqlite3 -y

In [ ]:
!sqlite3 --version

In [5]:
from helper import load_text_file
import json
import boto3
client = boto3.client('bedrock-runtime')

In [ ]:
# Studio Workshop does not have permission for Command+R
# This is left here for example
#def converse(session_id, input):
#  history = Chat.read(session_id)  
#
#  system_prompt = load_text_file("prompts/jp-teacher.txt")
#  model_id = 'cohere.command-r-v1:0'
#  prompt = f"{system_prompt}\n{history}\nUser: ${input}\nAssistant:"
#  resp = client.invoke_model(
#      modelId=model_id,
#      contentType='application/json',
#      accept='application/json',
#      body=json.dumps({
#        "message": prompt,
#        "max_tokens": 200,
#        "temperature": 1.0
#      })
#  )
#  body = json.loads(resp.get("body").read())
#  llm_response = body['text']
#
#  updated_history = f"{history}\n{llm_response}"
#  Chat.write(session_id,updated_history)
#
# return body

In [ ]:
model_id = 'amazon.titan-text-express-v1'

# Differen models are optimized for differents tasks.
# Titan Express is optimized for english which is why it not replying in the expected way.
system_prompt = load_text_file("prompts/jp-teacher.txt")
history = """
User: I want to eat at Kyoto Katsugyu in Toronto
Bot: That sounds like a great idea. Kyoto Katsugyu is a Japanese restaurant located in Toronto, Canada.
User: I wan't asking a question, that was the english text I wanted to translate to Japanese.
Bot: I apologize for the confusion. To clarify, would you like me to translate the English text into Japanese?
User: Where are my japanese clues?
"""

prompt = f"{system_prompt}\n{history}\nBot:"

resp = client.invoke_model(
  modelId=model_id,
  contentType='application/json',
  accept='application/json',
  body=json.dumps({
    "inputText": prompt
  })
)
print("Usage ====== \n")
print("Input Tokens: "+ resp['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-input-token-count'])
print("Output Tokens: " + resp['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-output-token-count'])
print("\nResponse ====== \n")
body = json.loads(resp.get("body").read())
print(body['results'][0]['outputText'])

In [ ]:
# Note: We will update our prompt document to tell is not to give us the direct answer.
messages = [
  {"role": "user", "content": [{"text": "I want to eat at Kyoto Katsugyu in Toronto"}]},
  {'role': 'assistant', 'content': [{'text': 'クルー:\n1) 私 (わたし) は トロントの 京都カツ牛 (きょうとかつぎゅう) で 食べたい (たべたい) と言っています。\n\n2) 京都カツ牛 は Toronto にある日本食レストランの名前です。\n\n3) 食べる (たべる) は「to eat」の意味の動詞の辞書形です。\n\nヒント:\n- 主語は私 (I)\n- トロントの〜で (in Toronto at〜) \n- レストラン名が出てきます\n- 動詞は「たい」形で欲求を表しています'}]},
  {"role": "user", "content": [{"text": "can you just tell me the translation?"}]}
]
system_prompt_text = load_text_file("prompts/jp-teacher.txt")
system_prompt = [{"text": system_prompt_text}]
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
resp = client.converse(
  modelId=model_id,
  messages=messages,
  system=system_prompt
)
print("Usage ====== \n")
print(resp['usage'])
print("\nResponse ====== \n")
text = resp['output']['message']['content'][0]['text']  
print(text)
print("\nMessage Object ====== \n")
msg = resp['output']['message']
print(msg)

In [29]:
from chat_history import Chat
def converse(session_id, input):
  history = Chat.read(session_id)  

  system_prompt = load_text_file("prompts/jp-teacher.txt")
  model_id = 'amazon.titan-text-express-v1'

  prompt = f"{system_prompt}\n{history}\nUser: ${input}\nBot:"
  resp = client.invoke_model(
      modelId=model_id,
      contentType='application/json',
      accept='application/json',
      body=json.dumps({
        "inputText": prompt
      })
  )
  body = json.loads(resp.get("body").read())
  llm_response = body['results'][0]['outputText']

  updated_history = f"{history}\n{llm_response}"
  Chat.write(session_id,updated_history)

  return body

In [ ]:
print(converse("food","Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

In [ ]:
print(converse("food","How do I conjugate taberu?"))

In [32]:
class JpAgent:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [
    
    ] # load with initial messages
  def chat(self, text):
    self.messages.append({"role": "user", "content": [{"text": text}]})
    system_prompt_text = load_text_file("prompts/jp-teacher.txt")
    system_prompt = [{"text": system_prompt_text}]
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse(
        modelId=model_id,
        messages=self.messages,
        system=system_prompt
    )
    text = resp['output']['message']['content'][0]['text']      
    self.messages.append({"role": "assistant", "content": [{"text": text}]})
    return text
agent = JpAgent()

In [ ]:
print(agent.chat("Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

In [ ]:
print(agent.chat("How do I conjugate taberu?"))

In [37]:
class JpAgentStream:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [] # load with initial messages
  def chat(self, text):
    self.messages.append({"role": "user", "content": [{"text": text}]})
    system_prompt_text = load_text_file("prompts/jp-teacher.txt")
    system_prompt = [{"text": system_prompt_text}]
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse_stream(
        modelId=model_id,
        messages=self.messages,
        system=system_prompt
    )
    text = self.stream(resp)
    self.messages.append({"role": "assistant", "content": [{"text": text}]})
    return text
  def stream(self,resp):  
    full_response = ""
    for event in resp.get('stream'):
        if 'messageStart' in event:
            print("Response started:", event['messageStart']['role'])
        elif 'contentBlockDelta' in event:
            chunk = event['contentBlockDelta']['delta'].get('text', '')
            full_response += chunk
            print(chunk, end='', flush=True)  # Print chunks as they arrive
        elif 'messageStop' in event:
            print("\n\nResponse completed. Stop reason:", event['messageStop']['stopReason'])
        elif 'metadata' in event:
            print("\nMetadata:", json.dumps(event['metadata'], indent=2))  
    return full_response          
stream_agent = JpAgentStream()

In [ ]:
stream_agent.chat("How do I conjugate taberu?")